In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

In [2]:
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
embeddings = OpenAIEmbeddings()

In [55]:
loader = TextLoader('video_transcription/final_file')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(docs)

In [3]:
from langchain.vectorstores import DeepLake
db = DeepLake(dataset_path="./my_deeplake/", embedding_function=embeddings, read_only=True)

./my_deeplake/ loaded successfully.



Deep Lake Dataset in ./my_deeplake/ already exists, loading from the storage


Dataset(path='./my_deeplake/', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (683, 1536)   None     None   
    ids      text     (683, 1)      str     None   
 metadata    json     (683, 1)      str     None   
   text      text     (683, 1)      str     None   


In [4]:
query = "Why is attention important in the attention model?"

In [5]:
ans = db.similarity_search(query,k=5)
context = ""
for i in range(5):
    context += ans[i].page_content
    context += "\n"

In [6]:
llm = OpenAI(temperature=0)
template="""Please use the following context to answer questions.
Context: {ans}
Question: {query}
Answer:"""

In [7]:
prompt = PromptTemplate(template=template, input_variables=["ans","query"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.predict(ans = context, query = query)

' Attention is important in the attention model because it acts as a communication mechanism between nodes in a directed graph. It allows nodes to look at each other and communicate information stored in vectors. Attention is also data dependent, meaning it takes into account the data stored at each node at any given point in time. Finally, attention is also used to create a scaled attention, which is a normalization technique to ensure that the inputs have a unit gaussian distribution.'